In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.manifold import TSNE
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
train_df = pd.read_csv("C:\\Users\\Lenovo\\OneDrive\\Desktop\\Hackathon\\Sensor\\Datasets\\train.csv")
test_df = pd.read_csv("C:\\Users\\Lenovo\\OneDrive\\Desktop\\Hackathon\\Sensor\\Datasets\\test.csv")
train_df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [24]:
gyro_features = [col for col in train_df.columns if 'Gyro' in col]
train_df.drop(columns=gyro_features, inplace=True)
test_df.drop(columns=gyro_features, inplace=True)
train_df

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyAccJerkMag-meanFreq(),fBodyBodyAccJerkMag-skewness(),fBodyBodyAccJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,0.346989,-0.516080,-0.802760,-0.112754,0.030400,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.532061,-0.624871,-0.900160,0.053477,-0.007435,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.660795,-0.724697,-0.928539,-0.118559,0.177899,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.678921,-0.701131,-0.909639,-0.036788,-0.012892,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.559058,-0.528901,-0.858933,0.123320,0.122542,-0.847865,0.185151,-0.043892,1,STANDING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.066650,-0.076714,-0.426588,-0.190437,0.829718,-0.791883,0.238604,0.049819,30,WALKING_UPSTAIRS
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,-0.193946,-0.151879,-0.509448,0.064907,0.875679,-0.771840,0.252676,0.050053,30,WALKING_UPSTAIRS
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,0.007099,0.331457,0.083984,0.052806,-0.266724,-0.779133,0.249145,0.040811,30,WALKING_UPSTAIRS
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,-0.109902,0.038175,-0.305554,-0.101360,0.700740,-0.785181,0.246432,0.025339,30,WALKING_UPSTAIRS


In [25]:
print("Remaining features:")
print(train_df.columns.tolist())

Remaining features:
['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X', 'tBodyAcc-max()-Y', 'tBodyAcc-max()-Z', 'tBodyAcc-min()-X', 'tBodyAcc-min()-Y', 'tBodyAcc-min()-Z', 'tBodyAcc-sma()', 'tBodyAcc-energy()-X', 'tBodyAcc-energy()-Y', 'tBodyAcc-energy()-Z', 'tBodyAcc-iqr()-X', 'tBodyAcc-iqr()-Y', 'tBodyAcc-iqr()-Z', 'tBodyAcc-entropy()-X', 'tBodyAcc-entropy()-Y', 'tBodyAcc-entropy()-Z', 'tBodyAcc-arCoeff()-X,1', 'tBodyAcc-arCoeff()-X,2', 'tBodyAcc-arCoeff()-X,3', 'tBodyAcc-arCoeff()-X,4', 'tBodyAcc-arCoeff()-Y,1', 'tBodyAcc-arCoeff()-Y,2', 'tBodyAcc-arCoeff()-Y,3', 'tBodyAcc-arCoeff()-Y,4', 'tBodyAcc-arCoeff()-Z,1', 'tBodyAcc-arCoeff()-Z,2', 'tBodyAcc-arCoeff()-Z,3', 'tBodyAcc-arCoeff()-Z,4', 'tBodyAcc-correlation()-X,Y', 'tBodyAcc-correlation()-X,Z', 'tBodyAcc-correlation()-Y,Z', 'tGravityAcc-mean()-X', 'tGravityAcc-mean()-Y', 'tGravi

In [26]:
X_train = train_df.drop(columns=['Activity'])
y_train = train_df['Activity']
X_test = test_df.drop(columns=['Activity'])
y_test = test_df['Activity']

In [27]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [28]:
y_test_encoded = label_encoder.transform(y_test)

In [29]:
pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [30]:
class LiquidLayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LiquidLayer, self).__init__()
        self.w = nn.Parameter(torch.randn(hidden_size, input_size))
        self.alpha = nn.Parameter(torch.rand(hidden_size))

    def forward(self, x):
        # Simple dynamic-like behavior
        z = torch.tanh(self.w @ x.T).T
        return z * self.alpha

class LiquidNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LiquidNeuralNetwork, self).__init__()
        self.liquid = LiquidLayer(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.liquid(x)
        x = self.fc(x)
        return x

In [31]:
X_train_tensor = torch.tensor(X_train_pca, dtype=torch.float32)
y_train_tensor = torch.tensor(pd.factorize(y_train)[0], dtype=torch.long)
X_test_tensor = torch.tensor(X_test_pca, dtype=torch.float32)
y_test_tensor = torch.tensor(pd.factorize(y_test)[0], dtype=torch.long)
class_names = pd.factorize(y_train)[1]

In [32]:
model = LiquidNeuralNetwork(input_size=50, hidden_size=64, num_classes=len(class_names))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/30, Loss: {loss.item():.4f}")

Epoch 5/30, Loss: 1.8183
Epoch 10/30, Loss: 1.7594
Epoch 15/30, Loss: 1.7043
Epoch 20/30, Loss: 1.6525
Epoch 25/30, Loss: 1.6039
Epoch 30/30, Loss: 1.5578
Epoch 35/30, Loss: 1.5141
Epoch 40/30, Loss: 1.4724
Epoch 45/30, Loss: 1.4325
Epoch 50/30, Loss: 1.3941
Epoch 55/30, Loss: 1.3571
Epoch 60/30, Loss: 1.3213
Epoch 65/30, Loss: 1.2867
Epoch 70/30, Loss: 1.2531
Epoch 75/30, Loss: 1.2205
Epoch 80/30, Loss: 1.1889
Epoch 85/30, Loss: 1.1580
Epoch 90/30, Loss: 1.1280
Epoch 95/30, Loss: 1.0986
Epoch 100/30, Loss: 1.0700


In [33]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    f1 = f1_score(y_test_tensor.numpy(), predicted.numpy(), average='weighted')
    print(f"\nAccuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nClassification Report:\n", classification_report(y_test_tensor, predicted, target_names=class_names))


Accuracy: 0.6542
F1 Score: 0.6569

Classification Report:
                     precision    recall  f1-score   support

          STANDING       0.56      0.64      0.60       532
           SITTING       0.51      0.52      0.52       491
            LAYING       0.85      0.69      0.76       537
           WALKING       0.62      0.75      0.68       496
WALKING_DOWNSTAIRS       0.76      0.60      0.67       420
  WALKING_UPSTAIRS       0.71      0.71      0.71       471

          accuracy                           0.65      2947
         macro avg       0.67      0.65      0.66      2947
      weighted avg       0.67      0.65      0.66      2947



In [34]:
def predict_activity(pca_input_vector):
    if len(pca_input_vector) != 50:
        raise ValueError("Input must be a 50-length PCA-transformed feature vector.")
    
    input_tensor = torch.tensor([pca_input_vector], dtype=torch.float32).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)
        predicted_label = torch.argmax(outputs, dim=1)

    return label_encoder.inverse_transform(predicted_label.cpu().numpy())[0]

In [35]:
custom_inputs = [
    # First input
    [0.77, -0.42, -0.04, -0.18, 0.04, 0.04, -0.04, -0.04, 0.04, -0.04,
     0.04, 0.04, -0.04, 0.04, -0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
     -0.04, 0.04, -0.04, 0.04, 0.04, -0.04, 0.04, -0.04, 0.04, 0.04,
     -0.04, 0.04, -0.04, 0.04, -0.04, 0.04, 0.04, -0.04, 0.04, -0.04,
     0.04, 0.04, -0.04, 0.04, -0.04, 0.04, 0.04, -0.04, 0.04, 0.04],
    
    # Second input
    [-7.141712188720703, -1.3797489404678345, -1.4743244647979736, -2.4590296745300293,
     0.6457638740539551, -2.2333781719207764, -0.8223496675491333, 0.027370205104351997,
     1.4218617677688599, -2.357050895690918, 0.34614503383636475, -0.059048812836408615,
     0.058187127113342285, -0.18423157930374146, 0.28658172488212585, -0.5763775706291199,
     0.3599643409252167, 0.35233256220817566, 0.49848732352256775, 0.32524818181991577,
     -0.28437328338623047, -0.027916865468025208, -0.06733408570289612, -0.07769753038883209,
     -0.2712530791759491, 0.31514036655426025, -0.33227652311325073, 0.2101690024137497,
     -0.2584279775619507, -0.2905064821243286, 0.20229914784431458, -0.2541504204273224,
     0.1986326277256012, -0.14399750530719757, 0.18883685767650604, -0.16962310671806335,
     -0.14629390835762024, 0.19317646324634552, 0.14222189784049988, -0.19504916667938232,
     -0.12527918815612793, 0.20916108787059784, -0.18004125356674194, -0.1210797056555748,
     -0.120667964220047, -0.11558661609888077, 0.15446516871452332, 0.1659548282623291,
     0.11470023542642593, -0.14440909028053284],
    
    # Third input
    [-6.98, -1.41, -1.56, -2.33, 0.72, -2.14, -0.79, 0.03, 1.35, -2.44,
     0.31, -0.07, 0.06, -0.19, 0.28, -0.59, 0.37, 0.33, 0.47, 0.31,
     -0.29, -0.03, -0.06, -0.08, -0.26, 0.32, -0.34, 0.22, -0.27, -0.28,
     0.20, -0.25, 0.21, -0.15, 0.19, -0.16, -0.14, 0.20, 0.14, -0.21,
     -0.12, 0.22, -0.19, -0.13, -0.12, -0.11, 0.16, 0.17, 0.11, -0.15],
    
    # Fourth input (all zeros)
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
     0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
     0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
     0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
     0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    
    # Fifth input (all ones)
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
     1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
     1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
]


In [44]:
random_index = random.randint(0, len(custom_inputs) - 1)
selected_input = custom_inputs[random_index]

In [45]:
predicted_activity = predict_activity(selected_input)
print("Selected input index:", random_index)
print("Predicted Activity:", predicted_activity)

Selected input index: 1
Predicted Activity: WALKING
